In [1]:
import tensorflow as tf

y = tf.Variable(1)
b = tf.identity(y)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(b,feed_dict={y:3})) #使用3 替换掉
    #tf.Variable(1)的输出结果，所以打印出来3 
    #feed_dict{y.name:3} 和上面写法等价

    print(sess.run(b))  #由于feed只在调用他的方法范围内有效，所以这个打印的结果是 1

3
1


In [5]:
import tensorflow as tf
state = tf.Variable(0.0,dtype=tf.float32)
one = tf.constant(1.0,dtype=tf.float32)
new_val = tf.add(state, one)
update = tf.assign(state, new_val) #返回tensor， 值为new_val
update2 = tf.assign(state, 10000)  #没有fetch，便没有执行
init = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init)
    for _ in range(3):
        #print(sess.run(update))
        print(sess.run(update2))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
10000.0
10000.0
10000.0


In [51]:
observations = tf.placeholder(tf.float32, None, name="input_x")
x = 1
state = tf.Variable(0.0,dtype=tf.float32)
test = tf.add(state, observations)

In [53]:
a = tf.zeros(shape=[1,2])

[[ 0.  0.]]


In [91]:
W = tf.Variable(tf.zeros(shape=[10,3]))
x = tf.Variable(tf.zeros(shape=[3,9]))
#Variable必须初始化以后才有具体的值。

W = tf.Variable([[ 1 , 2 , 3],[ 2 , 3 ,4]])
x = tf.Variable([[1],[2],[3]])
y = tf.placeholder(tf.int32 , [None, None])  # 输出占位符（预期输出）

In [92]:
a = tf.matmul(W,x )
init=tf.global_variables_initializer()
b = tf.matmul(W,y )
input = [ [ 1, 2, 3, 4, 5, 6],[ 1, 2, 3, 4, 5, 6],[ 1, 2, 3, 4, 5, 6] ]

In [93]:
with tf.Session() as sess:#创建sess
    sess.run(init)
    print(sess.run(a))
    print(sess.run(b,feed_dict={y:input}))

[[14]
 [20]]
[[ 6 12 18 24 30 36]
 [ 9 18 27 36 45 54]]


In [94]:
import numpy as np
import tensorflow as tf
import gym

In [97]:
D = 4  # 环境信息observation的维度D为4

tf.reset_default_graph()

#策略网络的具体结构。
#该网络将接受observation作为信息输入，最后输出一个概率值用以选择Action
#这里只有两个Action，向左施加力或者向右施加力，因此可以通过一个概率值决定

observations = tf.placeholder(tf.float32, [None, D], name="input_x")
#创建输入信息observations的placeholder其维度为D

#使用tf.contrib.layers.xavier_initializer方法初始化隐含层的权重W1，其维度为[D,H]
W1 = tf.get_variable("W1", shape=[D, H],initializer=tf.contrib.layers.xavier_initializer())

layer1 = tf.nn.relu(tf.matmul(observations, W1))
#接着使用tf.matmul将环境信息observations乘上W1再使用relu激活函数处理得到隐含层的输出layer1

#使用tf.contrib.layers.xavier_initializer方法初始化隐含层的权重W2，其维度为[H,1]
W2 = tf.get_variable("W2", shape=[H, 1],initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1, W2)
probability = tf.nn.sigmoid(score)
#将隐含层输出layer1乘以W2后，使用Sigmoid激活函数处理得到最后的输出概率


# From here we define the parts of the network needed for learning a good policy.
tvars = tf.trainable_variables()#获取策略网络中全部可训练的参数tvars
input_y = tf.placeholder(tf.float32, [None, 1], name="input_y")
advantages = tf.placeholder(tf.float32, name="reward_signal")
#定义人工设置的虚拟label的占位符input_y
#以及每个Action的潜在价值的占位符

#当Action取值为1的概率为probability(即策略网络输出的概率)
#当Action取值为0的概率为1-probability
#label取值与Action相反，即label=1-Action。
#当Action为1时，label为0，此时loglik=tf.log(probability),Action取值为1的概率的对数
#当Action为0时，label为1，此时loglik=tf.log(1-probability),Action取值为0的概率的对数
#所以，loglik其实就是当前Action对应的概率的对数
loglik = tf.log(input_y * (input_y - probability) + (1 - input_y) * (input_y + probability))

loss = -tf.reduce_mean(loglik * advantages)
#将loglik与潜在价值advanages相乘，并取负数作为损失，即优化目标
newGrads = tf.gradients(loss, tvars)
#使用tf.gradients求解模型参数关于loss的梯度

In [98]:
print(loglik)

Tensor("Log:0", shape=(?, 1), dtype=float32)


In [99]:
print(loss)

Tensor("Neg:0", dtype=float32)
